##### Import package

In [ ]:
import numpy as np 
import pandas as pd 
import xarray as xr 
import cartopy 
import glob
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import scipy 

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline, make_union
import warnings
import seaborn as sns
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.offsetbox as offsetbox
import seaborn.objects as so
from seaborn import axes_style
import matplotlib as mpl
import pickle
from matplotlib import cm

import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.gridspec as gridspec
import cartopy.feature as cfeature

from global_var_and_func import *
from plot_func import *


warnings.filterwarnings("ignore")
#from sklearn_xarray import Stacker, Select

##### Plots
1. modkern_modccf
2. cereskern_modccf
3. modkern_mmccf
4. isccpkern_modccf
5. patmoskern_modccf
6. modiskern_modccf
7. mmkern_modccf

- main: 1+2+3 with indiv model symbol
- supp A: 6+4+5+7 with indiv model symbol
- supp B: 1+2+3
- supp C: 6+4+5+7

In [ ]:
path = '/data/keeling/a/rytam2/ccf_model_spread/notebooks/data_smth/'
df_ikic = pd.read_csv(path+'df_smth.csv') #model kernels * model dCCF
df_ceres = pd.read_csv(path+'df_ceres_smth.csv') #ceres kernels * model CCF
df_modis = pd.read_csv(path+'df_modis_smth.csv') #modis kernels * model CCF
df_isccp = pd.read_csv(path+'df_isccp_smth.csv') #isccp kernels * model CCF
df_patmos = pd.read_csv(path+'df_patmosx_smth.csv') #patmosx kernels * model CCF
df_mmccf = pd.read_csv(path+'df_mmccf_smth.csv') #model kernels * model CCF average
df_mmkern = pd.read_csv(path+'df_mmkern_smth.csv')

# reshape dataframes for plotting
df_ikic,df_ceres,df_modis,df_isccp,df_patmos,df_mmccf,df_mmkern = melt(df_ikic,df_ceres,df_modis,df_isccp,df_patmos,df_mmccf,df_mmkern)

# Calculate staggered positions for each exp in each CCF var 
df_ikic = rename_exp(x_pos_calc(df_ikic))
df_ceres = rename_exp(x_pos_calc(df_ceres))
df_modis = rename_exp(x_pos_calc(df_modis))
df_isccp = rename_exp(x_pos_calc(df_isccp))
df_patmos = rename_exp(x_pos_calc(df_patmos))
df_mmccf = rename_exp(x_pos_calc(df_mmccf))
df_mmkern = rename_exp(x_pos_calc(df_mmkern))

#### Main Plot

In [ ]:
# Main Fig 
palette = {
    'AMIP': "#E69F00",
    'historical': "#0072B2",
    '4xCO2-fast': "#CC79A7",
    '4xCO2-slow': "#009E73"}

ccf = ['Total','SST','EIS','$T_{adv}$','RH','$\omega$','WS']
subplot_title=[r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{model} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$",
               r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{CERES} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$",
               r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{model} \frac{d\mathrm{CCF}}{dT_g}_{modavg} \right)$"]

df = rename_modname([df_ikic,df_ceres,df_mmccf])
summary = (stagger_not([mean_error_summary(i) for i in df],'no'))

fig, ax = plt.subplots(3,1,figsize=(12,18))#,sharex=True)
for i in range(3):
    sns.scatterplot(data=df[i], x='var_offset', y='dR/dT', hue='exp',style='model',ax=ax[i], alpha=0.7,palette=palette,s=80) #data
    ax[i].errorbar(summary[i]['var_offset'] , summary[i]['mean'], yerr=summary[i]['std'], fmt=' ', color='black', capsize=0,alpha=0.5) #statistics
    ax[i].scatter(summary[i]['var_offset'], summary[i]['mean'], color='red', zorder=3, marker='D')
    
    # Fig Aesthetics
    make_textbox(ax[i],textbox_labels[i])
    
    ax[i].set_xticks(ticks=list(x_pos_map.values()), labels=ccf,fontsize=13)
    if i ==2:
        ax[i].set_xlabel("$dR_{CCF}/dT_{g}$",fontsize=15)        
    else:
        ax[i].set_xlabel("")
    if i ==1:
        ax[i].legend(loc='upper left', bbox_to_anchor=(1.05, 1.11), borderaxespad=0.,prop={'size': 12},title_fontsize='12')
        ax[i].set_title
    else:
        ax[i].legend().remove()
        
    ax[i].set_ylabel("$Wm^{-2}K^{-1}$",fontsize=13)
    ax[i].set_ylim(-1.5,1.5)
    ax[i].tick_params(axis='y', labelsize=13)
    ax[i].grid(axis='y',alpha=0.3)
    ax[i].set_title(subplot_title[i],fontsize=16)

plt.savefig('dR_g_main_indvmod_1x3scatter_smth.png',dpi=400,bbox_inches='tight')

#### Supp Plot

###### Supp A

In [ ]:
#Supp A
subplot_title=[r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{MODIS} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$",
               r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{ISCCP} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$",
               r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{PATMOSX} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$",
              r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{modavg} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$"]

df = rename_modname([df_modis,df_isccp,df_patmos,df_mmkern])
summary = (stagger_not([mean_error_summary(i) for i in df],'no'))

fig, ax = plt.subplots(4,1,figsize=(12,16),sharex=True)
for i in range(4):
    sns.scatterplot(data=df[i], x='var_offset', y='dR/dT', hue='exp',style='model',ax=ax[i], alpha=0.7,palette=palette,s=80) #data
    ax[i].errorbar(summary[i]['var_offset'] , summary[i]['mean'], yerr=summary[i]['std'], fmt=' ', color='black', capsize=0,alpha=0.5) #statistics
    ax[i].scatter(summary[i]['var_offset'], summary[i]['mean'], color='red', zorder=3, marker='D')

    # Fig Aesthetics
    make_textbox(ax[i],textbox_labels[i])
    ax[i].legend( prop={'size': 12},title_fontsize='12')
    ax[i].set_xticks(ticks=list(x_pos_map.values()), labels=ccf,fontsize=13)
    if i ==3:
        ax[i].set_xlabel("$dR_{CCF}/dT_{g}$",fontsize=15)
    else:
        ax[i].set_xlabel("")

    if i ==1:
        ax[i].legend(loc='upper left', bbox_to_anchor=(1.05, 0.6), borderaxespad=0.,prop={'size': 12},title_fontsize='12')
        ax[i].set_title
    else:
        ax[i].legend().remove()


    ax[i].set_ylabel("$Wm^{-2}K^{-1}$",fontsize=13)
    ax[i].set_ylim(-1.5,1.5)
    ax[i].tick_params(axis='y', labelsize=13)
    ax[i].grid(axis='y',alpha=0.3)
    ax[i].set_title(subplot_title[i],fontsize=16)


plt.savefig('dR_g_supp_indvmod_1x4scatter_smth.png',dpi=400,bbox_inches='tight')

###### Supp B

In [ ]:
# Supp Fig B
subplot_title=[r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{model} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$",
               r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{CERES} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$",
               r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{model} \frac{d\mathrm{CCF}}{dT_g}_{modavg} \right)$"]

df = [df_ikic,df_ceres,df_mmccf]
summary = (stagger_not([mean_error_summary(i) for i in df],'no'))

fig, ax = plt.subplots(3,1,figsize=(12,15),sharex=True)
for i in range(3):
    sns.scatterplot(data=df[i], x='var_offset', y='dR/dT', hue='exp',ax=ax[i], alpha=0.7,palette=palette) #data
    ax[i].errorbar(summary[i]['var_offset'] , summary[i]['mean'], yerr=summary[i]['std'], fmt=' ', color='black', capsize=0,alpha=0.5) #statistics
    ax[i].scatter(summary[i]['var_offset'], summary[i]['mean'], color='red', zorder=3, marker='D')

    # Fig Aesthetics
    make_textbox(ax[i],textbox_labels[i])
    if i ==1:
        ax[i].legend(loc='upper left', bbox_to_anchor=(1.03, 0.65), borderaxespad=0.,prop={'size': 12},title_fontsize='12')
        ax[i].set_title
    else:
        ax[i].legend().remove()
    ax[i].set_xticks(ticks=list(x_pos_map.values()), labels=ccf,fontsize=13)
    if i ==2:
        ax[i].set_xlabel("$dR_{CCF}/dT_{g}$",fontsize=15)
    else:
        ax[i].set_xlabel("")
    ax[i].set_ylabel("$Wm^{-2}K^{-1}$",fontsize=13)
    ax[i].set_ylim(-1.5,1.5)
    ax[i].tick_params(axis='y', labelsize=13)
    ax[i].grid(axis='y',alpha=0.3)
    ax[i].set_title(subplot_title[i],fontsize=16)

plt.savefig('dR_g_supp_modm_1x3scatter_smth.png',dpi=400,bbox_inches='tight')

###### Supp C

In [ ]:
#Supp C
subplot_title=[r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{MODIS} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$",
               r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{ISCCP} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$",
               r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{PATMOSX} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$",
              r"$\left( \frac{\partial R}{\partial \mathrm{CCF}}_{modavg} \frac{d\mathrm{CCF}}{dT_g}_{model} \right)$"]

df = [df_modis,df_isccp,df_patmos,df_mmkern]
summary = (stagger_not([mean_error_summary(i) for i in df],'no'))

fig, ax = plt.subplots(4,1,figsize=(12,16),sharex=True)
for i in range(4):
    sns.scatterplot(data=df[i], x='var_offset', y='dR/dT', hue='exp',ax=ax[i], alpha=0.7,palette=palette) #data
    ax[i].errorbar(summary[i]['var_offset'] , summary[i]['mean'], yerr=summary[i]['std'], fmt=' ', color='black', capsize=0,alpha=0.5) #statistics
    ax[i].scatter(summary[i]['var_offset'], summary[i]['mean'], color='red', zorder=3, marker='D')

    # Fig Aesthetics
    make_textbox(ax[i],textbox_labels[i])
    if i ==1:
        ax[i].legend(loc='upper left', bbox_to_anchor=(1.05, 0.05), borderaxespad=0.,prop={'size': 12},title_fontsize='12')
        ax[i].set_title
    else:
        ax[i].legend().remove()
    ax[i].set_xticks(ticks=list(x_pos_map.values()), labels=ccf,fontsize=13)
    if i ==3:
        ax[i].set_xlabel("$dR_{CCF}/dT_{g}$",fontsize=15)
    else:
        ax[i].set_xlabel("")
    ax[i].set_ylabel("$Wm^{-2}K^{-1}$",fontsize=13)
    ax[i].set_ylim(-1.5,1.5)
    ax[i].tick_params(axis='y', labelsize=13)
    ax[i].grid(axis='y',alpha=0.3)
    ax[i].set_title(subplot_title[i],fontsize=16)

plt.savefig('dR_g_supp_modm_1x4scatter_smth.png',dpi=400,bbox_inches='tight')